In [62]:

import scapy.all as scp
import struct
import numpy as np
import matplotlib.pyplot as plt

from IrisBackendv3.data_standards import DataStandards
from IrisBackendv3.data_standards.logging import logger as DsLogger

from IrisBackendv3.codec.magic import Magic

from IrisBackendv3.utils.basic import print_bytearray_hex as printraw

In [63]:
DsLogger.setLevel('CRITICAL')
standards = DataStandards.build_standards()

## Settings

In [64]:
# Data Transport:
file = './test-data/Iris_FSWv1.0.0_210409_Telemetry.pcapng' # PCAP logs
protocol = scp.UDP # Protocol FSW is using to send data
port = 8080 # Port on the spacecraft FSW is sending data to

# Data Formatting Settings:
packetgap = 0 # number of packets to ignore at beginning of pcap
deadspace = 0 # number of bytes of deadspace at the beginning of the
endianness_code = "<" # < = little, > = big, ! = network

# Image Formatting Settings:
image_settings = {
    "width": 2592,
    "height": 1944
}

# Magics:
file_magic = bytearray([0xda,0xba,0xd0,0x00])
if endianness_code == '<':
    file_magic = bytearray([0x00,0xd0,0xba,0xda]) # Little Endian it.

## Decode First Packet
### Grab Packet

In [65]:
pcap = scp.rdpcap(file)

In [66]:
packets = list(filter(lambda x: x.dport == port, pcap[protocol][packetgap:]))
i = 0

In [67]:
packet = packets[i]

In [68]:
content = scp.raw(packet.getlayer(scp.Raw))[deadspace:]
# printraw(content)
packet_bytes = content

### Decode Common Packet Header

In [69]:
CPH_SIZE = 4
CPH = packet_bytes[:CPH_SIZE]
printraw(CPH)
cph_head, checksum = CPH[:-1], CPH[-1:]
seq_num, vlp_len = struct.unpack(endianness_code+' B H', cph_head)
print(seq_num, vlp_len, checksum)
#! TODO: Perform checksum check. (not impl. in FSW atm)
assert vlp_len == len(packet_bytes) - CPH_SIZE

00 cc 03 08 80
0 972 b'\x08\x80'


### Decode Variable Length Payload
#### Decode First Payload

In [1]:
# Extract the Variable Length Payload
VLP = packet_bytes[CPH_SIZE:]

# Parse VLP:

# Strip off the magic:
magic_bytes, VLP = VLP[:CPH_SIZE], VLP[CPH_SIZE:]
magic = Magic.decode(magic_bytes, byte_order='<') # treat as a byte array, so let it use network
magic

NameError: name 'packet_bytes' is not defined

In [71]:
if magic == Magic.TELEMETRY:
    # Grab Telemetry Header:
    module_id, channel_id, data_size, VLP = VLP[:1], VLP[1:2], VLP[2:4], VLP[4:]
    print(module_id, channel_id, data_size)
    # Unpack + Format Telemetry Header:
    module_id = struct.unpack(endianness_code+'B', module_id)[0] << 8
    channel_id = struct.unpack(endianness_code+'B', channel_id)[0]
    #! TODO: Not impl. in fsw. Once is, perform check against expected size.
    # ... Rn, replace w/expected size
    data_size = struct.unpack(endianness_code+'H', data_size)[0]
    try:
        module = standards.modules[module_id]
    except KeyError:
        logger.
    exp_data_size = standards.modules[module_id] 

    print(hex(module_id), hex(channel_id), data_size)
    # data, timestamp, VLP = VLP[:data_size], VLP[data_size:data_size+4], VLP[data_size+4:]

b'\x0f' b'\x00' b'\x00\x00'
0xf00 0x0 0


In [82]:
standards.modules['Imu'].telemetry

[((['XAcc'], 0), Channel[0]::XAcc: int16), ((['YAcc'], 1), Channel[1]::YAcc: int16), ((['ZAcc'], 2), Channel[2]::ZAcc: int16), ((['XAng'], 3), Channel[3]::XAng: uint16), ((['YAng'], 4), Channel[4]::YAng: uint16), ((['ZAng'], 5), Channel[5]::ZAng: uint16)]

In [81]:
[t.datatype for t in standards.modules['Imu'].telemetry.vals]

[<FswDataType.I16: 'int16'>,
 <FswDataType.I16: 'int16'>,
 <FswDataType.I16: 'int16'>,
 <FswDataType.U16: 'uint16'>,
 <FswDataType.U16: 'uint16'>,
 <FswDataType.U16: 'uint16'>]